In [1]:
wsi_csv = '/media/yanis/LaCie/Final_execution_files/train/process_list_autogen.csv'
wsi_path = '/media/yanis/LaCie/final_training_val_set/'
patches_path = '/media/yanis/LaCie/Final_execution_files/train/patches/'
output_path = '/media/yanis/LaCie/Final_execution_files/train/features2/'

In [2]:
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
from torchvision import transforms
import h5py

class WSI_Dataset(Dataset):

	def __init__(self, csv_path):
		self.df = pd.read_csv(csv_path)
	
	def __len__(self):
		return len(self.df)

	def __getitem__(self, idx):
		return self.df['slide_id'][idx]
        
class Patches_Dataset(Dataset):
	def __init__(self,
		file_path,
		wsi,
		pretrained=False,
		custom_transforms=None,
		custom_downsample=1,
		target_patch_size=-1
		):
		self.pretrained=pretrained
		self.wsi = wsi
		self.roi_transforms = transforms.Compose([transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.485, 0.456, 0.406), 
                                                                       std = (0.229, 0.224, 0.225))])
		self.file_path = file_path
		with h5py.File(self.file_path, "r") as f:
			dset = f['coords']
			self.patch_level = f['coords'].attrs['patch_level']
			self.patch_size = f['coords'].attrs['patch_size']
			self.length = len(dset)
			if target_patch_size > 0:
				self.target_patch_size = (target_patch_size, ) * 2
			elif custom_downsample > 1:
				self.target_patch_size = (self.patch_size // custom_downsample, ) * 2
			else:
				self.target_patch_size = None
		self.summary()

	def __len__(self):
		return self.length

	def summary(self):
		hdf5_file = h5py.File(self.file_path, "r")
		dset = hdf5_file['coords']
		for name, value in dset.attrs.items():
			print(name, value)

	def __getitem__(self, idx):
		with h5py.File(self.file_path,'r') as hdf5_file:
			coord = hdf5_file['coords'][idx]
		img = self.wsi.read_region(coord, self.patch_level, (self.patch_size, self.patch_size)).convert('RGB')

		if self.target_patch_size is not None:
			img = img.resize(self.target_patch_size)
		img = self.roi_transforms(img).unsqueeze(0)
		return img, coord
        
def collate_features(batch):
	img = torch.cat([item[0] for item in batch], dim = 0)
	coords = np.vstack([item[1] for item in batch])
	return [img, coords]
    

In [3]:
def save_hdf5(output_path, asset_dict, attr_dict= None, mode='a'):
    file = h5py.File(output_path, mode)
    for key, val in asset_dict.items():
        data_shape = val.shape
        if key not in file:
            data_type = val.dtype
            chunk_shape = (1, ) + data_shape[1:]
            maxshape = (None, ) + data_shape[1:]
            dset = file.create_dataset(key, shape=data_shape, maxshape=maxshape, chunks=chunk_shape, dtype=data_type)
            dset[:] = val
            if attr_dict is not None:
                if key in attr_dict.keys():
                    for attr_key, attr_val in attr_dict[key].items():
                        dset.attrs[attr_key] = attr_val
        else:
            dset = file[key]
            dset.resize(len(dset) + data_shape[0], axis=0)
            dset[-data_shape[0]:] = val
    file.close()
    return output_path

The real time isn't showed, it took way longer (hours)

In [4]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from models.resnet_custom import resnet50_baseline
import openslide

wsi_bag = WSI_Dataset(wsi_csv)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = resnet50_baseline(pretrained=True)
model = model.to(device)
mode = 'w'

for item in wsi_bag.df['slide_id']:
    wsi = openslide.open_slide(wsi_path+item+'.tif')
    patches = Patches_Dataset(patches_path+item+'.h5', wsi, True, 1, 256)
    patches_loader = DataLoader(patches, 256, {'num_workers': 4, 'pin_memory': True}, collate_fn=collate_features)
    print('Bag : {} - Processing {} batches in total'.format(item,len(patches_loader)))
    for i, (batch_images,batch_coords) in enumerate(tqdm(patches_loader)):
        with torch.no_grad():
            #if i % 10 == 0:
            	#print('batch {}/{}, {} files processed'.format(i, len(patches_loader), i * 256))
            batch_images = batch_images.to(device, non_blocking=True)
            features = model(batch_images)
            features = features.cpu().numpy()
            outputs = {'features': features, 'coords': batch_coords}
            save_hdf5(output_path+item+'_features.h5', outputs, None, mode=mode)
            mode = 'a'

    file = h5py.File(output_path+item+'_features.h5', "r")
    
    features = file['features'][:]
    features = torch.from_numpy(features)
    torch.save(features, output_path+item+'_features.pt')

print("The features' computation is done.")


downsample [1. 1.]
downsampled_level_dim [ 97792 221184]
level_dim [ 97792 221184]
name normal_001
patch_level 0
patch_size 256
save_path ./project/execution_files/train/patches
Bag : normal_001 - Processing 32 batches in total


100%|███████████████████████████████████████████| 32/32 [00:41<00:00,  1.30s/it]


downsample [1. 1.]
downsampled_level_dim [ 97792 221184]
level_dim [ 97792 221184]
name normal_002
patch_level 0
patch_size 256
save_path ./project/execution_files/train/patches
Bag : normal_002 - Processing 62 batches in total


100%|███████████████████████████████████████████| 62/62 [01:03<00:00,  1.03s/it]


downsample [1. 1.]
downsampled_level_dim [ 97792 221184]
level_dim [ 97792 221184]
name normal_003
patch_level 0
patch_size 256
save_path ./project/execution_files/train/patches
Bag : normal_003 - Processing 93 batches in total


...

downsample [1. 1.]
downsampled_level_dim [ 97792 221184]
level_dim [ 97792 221184]
name tumor_016
patch_level 0
patch_size 256
save_path ./project/execution_files/train/patches
Bag : tumor_016 - Processing 68 batches in total


100%|███████████████████████████████████████████| 68/68 [01:46<00:00,  1.56s/it]


downsample [1. 1.]
downsampled_level_dim [ 97792 219648]
level_dim [ 97792 219648]
name tumor_017
patch_level 0
patch_size 256
save_path ./project/execution_files/train/patches
Bag : tumor_017 - Processing 141 batches in total


100%|█████████████████████████████████████████| 141/141 [03:13<00:00,  1.37s/it]


The features' computation is done.
